In [1]:
from distributed import Client

In [2]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:58699 Dashboard: http://127.0.0.1:58700/status,Cluster Workers: 8 Cores: 8 Memory: 17.18 GB


In [104]:
def ring_all_reduce(n):
    return {k: (k + 1) % n for k in range(n)}
    return {(k + 1) % n: k for k in range(n)}
    
ring_all_reduce(5)

{0: 1, 1: 2, 2: 3, 3: 4, 4: 0}

In [139]:
class Worker:
    def __init__(self, model, comm, worker_id=0):
        self.model = model
        self._worker_id = worker_id
        self.models = {}
        
    @property
    def worker_id(self):
        return self._worker_id
    @property
    def model_(self):
        return self.model
    
    def send(self, worker, final=False):
        worker.recv(self.model, self.worker_id, final=final)
        return True
        
    def recv(self, model, worker_id, final=False):
        self.models[worker_id] = model
        if not final:
            self.model = sum(self.models.values()) + self.model
        else:
            self.model = model
        return True


In [140]:
n = 5
comms = ring_all_reduce(n)
futures = [client.submit(Worker, (i + 1) * 10, comms[i], worker_id=i, actor=True)
           for i in range(n)]
w = client.gather(futures)
w

[<Actor: Worker, key=Worker-2b76407e-b330-4a41-be66-702d26058d66>,
 <Actor: Worker, key=Worker-ab06924e-88f3-4d5e-a291-b2d70367ced1>,
 <Actor: Worker, key=Worker-d12301ef-22b8-4191-84c4-30a87fca9fa3>,
 <Actor: Worker, key=Worker-8c785487-e0ab-475c-ab1f-e4b8b9f8701f>,
 <Actor: Worker, key=Worker-385c176a-a2d1-44ba-a422-a313896830ab>]

In [141]:
workers = {worker.worker_id: worker for worker in w}

futures = []
for id_, worker in workers.items():
    send_to = comms[id_]
    futures += [worker.send(workers[send_to])]
sent = [f.result() for f in futures]
models = {k: worker.model_ for k, worker in workers.items()}
models

{0: 60, 1: 30, 2: 60, 3: 70, 4: 90}

In [144]:
complete_worker = max(models.keys())
final_worker = workers[complete_worker]

In [145]:
futures = []
for id_, worker in workers.items():
    futures += [final_worker.send(worker, final=True)]
[f.result() for f in futures]

[True, True, True, True, True]

In [147]:
models = {k: worker.model_ for k, worker in workers.items()}
models

{0: 90, 1: 90, 2: 90, 3: 90, 4: 90}